## 安装package

<font color=#FF000 >安装后需要在 jupyter 页面重启  kernel，才会生效</font>  
<font color=#FF000 >必须！python3.7的kernel, 否则会遇到组件兼容性问题</font>

In [ ]:
#（建议）安装最新版本
!pip install --verbose -U fast-causal-inference

## 初始化fast_causal_inference

In [2]:
import fast_causal_inference
#支持业务多租户安全隔离，请首先初始化租户信息才可进行后续使用
fast_causal_inference.set_default(tenant_id='${guest_tenant_id}', tenant_secret_key='${guest_tenant_secret_key}')
sql_instance = fast_causal_inference.create()

### 创建/删除  clickhouse表(视图)
<font color=#FF000 >建议用户基于此function在clickhouse中创建/删除表 </font>  

#### 创建表： 写完整查询SQL

In [3]:
fast_causal_inference.clickhouse_create_view(
                            clickhouse_view_name="a20231018_bearlyhuang4", 
                            sql_statement="""
                            WITH (
                                    SELECT
                                            OlsState(y,x1,x2) AS model
                                    FROM
                                            test_data_small
                                   ) AS model
                              SELECT
                                    evalMLMethod (model,x1,x2) AS y_pre
                              FROM
                                    test_data_small
                              limit 10""",
                              is_sql_complete=True, is_use_local=False)
"""
    创建实验指标明细视图:sql_statement, sql_table_name, sql_where, sql_group_by, sql_limit 会组成完整sql
参数：
    :param clickhouse_view_name: clikchouse视图/物理表名称
    :param sql_statement:  查询子语句 如果is_sql_complete为True, 则为创建表的完整查询
    :param is_sql_complete(默认False) sql_statement字段是否提供的是完整的sql声明,  如果为True，则不需要额外提供sql_table_name, sql_where, sql_group_by, sql_limit 
"""

running, please wait
[2023-10-18 21:56:50] clickhouse.py line:532 INFO - 3
donetime cost: 1.3996154814958572 Seconds


'\n    创建实验指标明细视图:sql_statement, sql_table_name, sql_where, sql_group_by, sql_limit 会组成完整sql\n参数：\n    :param clickhouse_view_name: clikchouse视图/物理表名称\n    :param sql_statement:  查询子语句 如果is_sql_complete为True, 则为创建表的完整查询\n    :param is_sql_complete(默认False) sql_statement字段是否提供的是完整的sql声明,  如果为True，则不需要额外提供sql_table_name, sql_where, sql_group_by, sql_limit \n'

#### 删除表

In [4]:
fast_causal_inference.clickhouse_drop_view(clickhouse_view_name="a20231018_bearlyhuang4") 
"""
    在集群删除视图/物理表, 删除操作需谨慎
    :param clickhouse_view_name:  视图或物理表名称
"""

running, please wait
donetime cost: 0.4244374483823776 Seconds


'\n    在集群删除视图/物理表, 删除操作需谨慎\n    :param clickhouse_view_name:  视图或物理表名称\n'

## 将外部数据导到clickhouse里
如果只是想试用，不需要导入自己的数据做分析，可以跳过这一步

#### tdw数据: thive表 >> clickhouse（通过spark）

In [ ]:
# 申请sparkSession
spark = fast_causal_inference.get_spark_session(group_id='', gaia_id='') # group_id,gaia_id 参考 notebook上默认文件！Spark 资源池.html
# spark = fast_causal_inference.get_spark_session(group_id='', gaia_id='',driver_cores=4, driver_memory="8g", executor_cores=2, executor_memory="10g", spark_eventLog_compress="false")

In [ ]:
# thive dataframe
from pytoolkit import TDWSQLProvider
tdw= TDWSQLProvider(spark, 
                    user='tdw_rtx',# tdw账号："tdw_rtx"
                    passwd='', #查看tdw账号密码 ${tdw_password}
                    db='') # 查询的数据库
thive_df=tdw.table(tblName='', #表名
                   priParts=['p_']) #如果是分区表需要加上所取分区： “p_分区名称” 
# 如果该表已经存在，需要用clickhouse_drop_view将其删除
# fast_causal_inference.clickhouse_drop_view(clickhouse_view_name="test_data_small_20230920_thive")
fast_causal_inference.dataframe_2_clickhouse(dataframe=thive_df, 
                                             clickhouse_table_name="test_data_small_20231017_thive")

#### tdw数据: iceberge表 >> clickhouse（通过spark）

In [ ]:
# 申请sparkSession 需要指定cmk, 下载 ${tdw_cmk} 到notebook
# spark.stop()
spark = fast_causal_inference.get_spark_session(group_id='', gaia_id='', cmk="/home/bearlyhuang/bearlyhuang.cmk")

In [ ]:
# iceberg dataframe
iceberg_df = spark.read.format("iceberg").load("").limit(100) # 数据库.表名称
# 如果该表已经存在，需要用clickhouse_drop_view将其删除
# fast_causal_inference.clickhouse_drop_view(clickhouse_view_name="test_data_small_20231017_iceberg")
fast_causal_inference.dataframe_2_clickhouse(dataframe=iceberg_df, 
                                             clickhouse_table_name="test_data_small_20231017_iceberg")

#### clickhouse >> spark 

In [ ]:
df = fast_causal_inference.clickhouse_2_dataframe(spark, clickhouse_table_name='test_data_small_20230920_thive')
df.show()

#### 本地数据表csv <<>> clickhouse

In [2]:
# csv >> clickhouse    csv文件请将第一行应表示为列名
fast_causal_inference.csv_2_clickhouse(csv_file_abs_path="test_load.csv", clickhouse_table_name="test_load3")

running, please wait
donetime cost: 0.5833354592323303 Seconds


In [3]:
# clickhouse >> csv
fast_causal_inference.clickhouse_2_csv(csv_file_abs_path="test_load2.csv", clickhouse_table_name="test_load3")

running, please wait
donetime cost: 0.749113954603672 Seconds
